In [6]:
%load_ext autoreload
%autoreload 2

import logging
logging.getLogger("querychains").setLevel(logging.DEBUG)
import querychains
from querychains import with_context, Context, FileStorage, start_server, add_tag, Tag

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
storage = FileStorage("data")
storage

<FileStorage directory='/home/ada/projects/querychain/data'>

In [ ]:
# Register context into storage

with Context("root1", storage=storage):
    pass


In [ ]:
# Manual writing context into storage

with Context("root2") as c:
    pass

storage.write_context(c)

In [ ]:
# Composing directory structure with contexts

with Context("root3", storage=storage, directory=True) as root:
    with Context("first child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass
    with Context("second child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass

import pathlib
list(pathlib.Path(f"data/{root.uid}.ctx").rglob("*"))

In [10]:
# Tags

with Context("root", tags=["tag1", "tag2"], storage=storage) as c:
    c.add_tag("exp1")  # Add to a context dynammicaly
    with Context("something"):
        add_tag("hi!")  # Add to the current context
        add_tag(Tag("success!", color="lightgreen"))  # Added colored tag

DEBUG:querychains:Context None inputs=None
DEBUG:querychains: Context None inputs=None
DEBUG:querychains: -> OK   None result='None'
DEBUG:querychains:-> OK   None result='None'


INFO:     127.0.0.1:35198 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:35198 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:35198 - "OPTIONS /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:35198 - "OPTIONS /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:35214 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:35214 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:35214 - "GET /contexts/uid/2023-06-26T17%3A34%3A47-Long_running-ku6doh HTTP/1.1" 200 OK
INFO:     127.0.0.1:35214 - "GET /contexts/uid/2023-06-26T17%3A34%3A47-Long_running-ku6doh HTTP/1.1" 200 OK
INFO:     127.0.0.1:35214 - "GET /contexts/uid/2023-06-29T12%3A10%3A47-root-LSFxN1 HTTP/1.1" 200 OK


In [4]:
# Running data browser over storage

server = start_server(storage=storage, port=5000)

INFO:     Started server process [32793]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


In [ ]:
# Long running cell, you can observe it in browser in running state

with Context("Long running", storage=storage):
    with Context("Child1"):
        import time
        time.sleep(10)